<a href="https://colab.research.google.com/github/Nanangk/Transfer_Learning/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Transfer Learning

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

In [ ]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 800
nb_validation_samples = 240
epochs = 50
batch_size = 16

In [ ]:
# TensorBoard Callbacks
callbacks = TensorBoard(log_dir='./Graph')

In [ ]:
# Build VGG16
model = applications.VGG16(include_top=False, weights='imagenet')

In [ ]:
# Training Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Rescale Testing Data
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# Train Data Generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
train_features = model.predict_generator(
    train_generator, nb_train_samples // batch_size, verbose=1)
np.save('train_features.npy', train_features)

FileNotFoundError: ignored

In [ ]:
# Testing Data Generator
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
validation_features = model.predict_generator(
    validation_generator, nb_train_samples // batch_size, verbose=1)
np.save('val_features.npy', validation_features)

FileNotFoundError: ignored

In [ ]:
# Build Train Data
train_data = np.load('train_features.npy')
train_labels = np.array(
    [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

# Build Validation Data
validation_data = np.load('val_features.npy')
validation_labels = np.array(
    [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

In [ ]:
# Build FC Layer
fc_model = Sequential()
fc_model.add(Flatten(input_shape=train_data.shape[1:]))
fc_model.add(Dense(32, activation='relu'))
fc_model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Adam Optimizer and Cross Entropy Loss
adam = Adam(lr=0.0001)
fc_model.compile(optimizer=adam,
                loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
fc_model.fit(train_data, train_labels,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(validation_data, validation_labels), 
            callbacks=[callbacks])
            
fc_model.save_weights('vgg16.py')